# Work Environment & Data Preparation Template

**Project:** Identify Customer Segments for Online Retail using K-Means Clustering

**Name:** Gali Tarun Roshan

**Role:** Data Science Intern



In [ ]:
# Basic imports for data prep and EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Display settings
pd.set_option('display.max_columns', 200)
pd.set_option('display.precision', 3)

print("Libraries imported successfully")


## 2. Load the Data

Place your dataset in a `data/` folder (e.g., `data/online_retail.csv`).

Supported formats: CSV, Excel, or SQL.


In [ ]:
# Example: load CSV
# df = pd.read_csv('data/online_retail.csv', encoding='latin1')
# For Excel: df = pd.read_excel('data/online_retail.xlsx', sheet_name='Sheet1')

# Placeholder: create a small sample DataFrame if file not present
data = {
    'CustomerID': [12345, 12346, 12347, 12348, 12345],
    'InvoiceNo': ['A001', 'A002', 'A003', 'A004', 'A005'],
    'InvoiceDate': pd.to_datetime(['2020-01-05','2020-01-07','2020-02-01','2020-02-05','2020-03-01']),
    'Quantity': [10, 5, 2, 1, 3],
    'UnitPrice': [2.5, 5.0, 1.25, 10.0, 2.5],
    'Country': ['United Kingdom','France','United Kingdom','Germany','United Kingdom']
}
df = pd.DataFrame(data)
df.head()

## 3. Understand the Data

Run basic checks: head, info, describe, nulls, duplicates.

In [ ]:
# Basic EDA
print('Shape:', df.shape)
display(df.head())
display(df.info())
display(df.describe(include='all'))

# Missing values & duplicates
print('\nMissing values per column:\n', df.isnull().sum())
print('\nDuplicate rows:', df.duplicated().sum())

## 4. Clean the Data

Handle missing values, duplicates, and data types.

In [ ]:
# Drop duplicates example
df_clean = df.drop_duplicates().copy()

# Convert types if necessary (example)
df_clean['CustomerID'] = df_clean['CustomerID'].astype(str)

# Handle missing values (example strategy)
# imputer = SimpleImputer(strategy='median')
# df_clean['Quantity'] = imputer.fit_transform(df_clean[['Quantity']])

df_clean.info()

## 5. Transform the Data

Scaling and encoding examples.

In [ ]:
# Feature: TotalPrice
df_clean['TotalPrice'] = df_clean['Quantity'] * df_clean['UnitPrice']

# Scaling numeric features (example)
numeric_features = ['Quantity', 'UnitPrice', 'TotalPrice']
scaler = StandardScaler()
df_clean[numeric_features] = scaler.fit_transform(df_clean[numeric_features])

# Encoding categorical features (example)
df_clean = pd.get_dummies(df_clean, columns=['Country'], drop_first=True)

df_clean.head()

## 6. Feature Engineering

Example: Create RFM features

In [ ]:
# Example RFM features for customer segmentation
# For real data, ensure InvoiceDate is datetime and CustomerID exists
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalPrice': lambda x: (x*1).sum() if 'TotalPrice' in df.columns else (df.loc[x.index,'Quantity']*df.loc[x.index,'UnitPrice']).sum()
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
rfm['CustomerID'] = rfm['CustomerID'].astype(str)
display(rfm.head())

## 7. Feature Selection & Save Cleaned Data

Select features for clustering and save normalized data.

In [ ]:
# Select features (example)
features = ['Recency', 'Frequency', 'Monetary']
rfm_selected = rfm[features].fillna(0)

# Normalize
mms = MinMaxScaler()
rfm_norm = pd.DataFrame(mms.fit_transform(rfm_selected), columns=features)
rfm_norm.head()

# Save to CSV for submission
rfm_norm.to_csv('cleaned_normalized_features_Gali_Tarun_Roshan.csv', index=False)
print('Saved cleaned normalized features to cleaned_normalized_features_Gali_Tarun_Roshan.csv')